In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
tqdm.pandas()
%matplotlib inline
import string

In [2]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/lyrics.csv").sample(frac=0.1,replace=False,random_state=101)


In [3]:
df = df.drop(columns=['Unnamed: 0'])


In [4]:
import re
from nltk.corpus import stopwords
from tqdm import tqdm

stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

df['Lyric'] = df['Lyric'].progress_apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stop_words | punctuation)]))


100%|██████████| 68283/68283 [00:55<00:00, 1237.27it/s]


In [5]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
from sklearn.metrics import accuracy_score


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Define a function to create a progress bar
def tqdm_fit_transform(corpus):
    total = len(corpus)
    with tqdm(total=total, desc="Processing") as pbar:
        X = tfidf_vectorizer.fit_transform(corpus)
        pbar.update(total)
    return X

# Fit and transform the 'Lyric' column to convert text into numerical features with a progress bar
X = tqdm_fit_transform(df['Lyric'])

# 'genre' column will be your target variable
y = df['genre']


Processing: 100%|██████████| 68283/68283 [00:11<00:00, 6091.51it/s]


In [8]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now you can proceed to train your machine learning model using X_train and y_train,
# and evaluate its performance on X_test and y_test.


In [12]:
! pip install tqdm

In [10]:
from sklearn.ensemble import RandomForestClassifier

# Define and initialize the random forest classifier
rf_classifier = RandomForestClassifier()

In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [90, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 0.5],
    'criterion': ['gini', 'entropy'],
    'class_weight': [None, 'balanced'],
    'oob_score': [False, True]
}

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")

best_rf_classifier = grid_search.best_estimator_
y_pred = best_rf_classifier.predict(X_test)


KeyboardInterrupt: 

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Initialize the classifier with your chosen parameters
rf_classifier = RandomForestClassifier(
    n_estimators=100,       # Number of trees
    max_depth=5,           # Maximum depth of each tree
    min_samples_split=5,    # Minimum number of samples required to split an internal node
    min_samples_leaf=5,     # Minimum number of samples required to be at a leaf node
    max_features='sqrt',    # Number of features to consider when looking for the best split
    criterion='gini',       # The function to measure the quality of a split
    class_weight=None,      # Weights associated with classes
    oob_score=False,        # Whether to use out-of-bag samples to estimate the generalization accuracy
    random_state=42,        # Ensures reproducibility
    verbose=0               # Set verbose to 0 to not conflict with tqdm
)

# Wrap the training process with tqdm for a progress bar
for i in tqdm(range(100), desc="Training Progress"):
    # Train the classifier on the training data
    rf_classifier.fit(X_train, y_train)

# Predict the labels for the testing set
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")


Training Progress: 100%|██████████| 100/100 [17:24<00:00, 10.45s/it]


Model Accuracy: 0.5270557223401918
